In [9]:
# prompt: please install all the below using pip

!pip install numpy pandas matplotlib seaborn scikit-learn tensorflow keras clean_text pandarallel


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.7 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=eb3cea677b766243249202b2e24810273e8a62aa283329be3eff5818a70ea1c4
  Stored in directory: /root/.cache/pip/wheels/b9/c6/5a/829298789e94348b81af52ab42c19d49da007306bbcc983827
Successfully built pandarallel


In [10]:
import nltk
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.lm import Vocabulary
from sklearn.model_selection import train_test_split
from cleantext import clean
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

import os
IN_COLAB = False
if os.getenv("COLAB_RELEASE_TAG"):
   IN_COLAB = True


INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [12]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Part 1

### Task 1

In [13]:
#load data
dataPath = "../data/"
if IN_COLAB:
  dataPath = "/content/drive/MyDrive/"
nsdf = pd.read_csv(dataPath + "news_sample.csv")
nsdf = nsdf.reset_index(drop=True)  # Reset index??
nsdf_raw = nsdf
print(nsdf.info())   # Check column types and missing values



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        250 non-null    int64  
 1   id                250 non-null    int64  
 2   domain            250 non-null    object 
 3   type              238 non-null    object 
 4   url               250 non-null    object 
 5   content           250 non-null    object 
 6   scraped_at        250 non-null    object 
 7   inserted_at       250 non-null    object 
 8   updated_at        250 non-null    object 
 9   title             250 non-null    object 
 10  authors           170 non-null    object 
 11  keywords          0 non-null      float64
 12  meta_keywords     250 non-null    object 
 13  meta_description  54 non-null     object 
 14  tags              27 non-null     object 
 15  summary           0 non-null      float64
dtypes: float64(2), int64(2), object(12)
memory u

In [14]:
# unique lable values
unique_values = nsdf['type'].unique()
print(unique_values)

['unreliable' 'fake' 'clickbait' 'conspiracy' 'reliable' 'bias' 'hate'
 'junksci' 'political' nan 'unknown']


In [15]:
#nan and unknown removed as they seem useless when training a classifier
nsdf = nsdf.dropna(subset=['type'])
nsdf = nsdf.loc[nsdf['type']!='unknown']
newunique_values = nsdf['type'].unique()
print(newunique_values)


['unreliable' 'fake' 'clickbait' 'conspiracy' 'reliable' 'bias' 'hate'
 'junksci' 'political']


Cleaning and Preprocessing

In [16]:
def cleanText(data, column):
    def clean_text_help(text):
        if isinstance(text, str):
            # Remove excess whitespace
            text = re.sub(r"\s+", " ", text).strip()
            #replace dates
            text = re.sub(r"(0[1-9]|[1-2][0-9]|3[0-1])[-/.]?(0[1-9]|1[0-2])[-/.]?([0-9]{2}|[0-9]{4})", "<DATE>", text)  # Replace date type 1
            text = re.sub(r"(0[1-9]|[1-2][0-9]|3[0-1])\s([A-Za-z]{3})\s([0-9]{2}|[0-9]{4})", "<DATE>", text)  # Replace date type 2
            return clean(text, lower=True, no_line_breaks=True, no_punct=True, replace_with_url="<URL>", replace_with_email="<EMAIL>", replace_with_number="<NUM>", replace_with_digit="<NUM>")
        return text  # Return unchanged if not a string
    data[column] = data[column].parallel_apply(clean_text_help)  # Apply function
    return data

In [17]:
#Tokenize the text function
def tokenizeText(data, column):
    def tokenize_text_help(text):
        if isinstance(text, str):
            return word_tokenize(text)
        return text  # Return unchanged if not a string
    data[column] = data[column].parallel_apply(tokenize_text_help)  # Apply function
    return data

In [18]:
#function for removeing stopwords
def remove_stopwords(data, column):
    def remove_stopwords_help(text):
        stop_words = set(stopwords.words('english'))  # Load stopwords
        if isinstance(text, str):
            return [word for word in text.at[0, 'content'] if not word.lower() in stop_words]
        return text  # Return unchanged if not a string
    data[column] = data[column].parallel_apply(remove_stopwords_help)  # Apply function
    return data

In [19]:
#funtion for populating vocabulary
def populate_vocabulary(data):
    N = data.shape[0]  # Get the number of rows
    allWords = []
    for i in range(N):
        if isinstance(data.at[i, 'content'], str):  # Ensure it's a string
            allWords.append(data.at[i, 'content'])
    return Vocabulary(allWords, unk_cutoff=2)

#langsom kørertid men kunne ikke finde ud af det med apply. Nogne med en god ide??
# Can just use value_vounts()


In [20]:
#making frequency dictionary
def getFrequency(data, column, preTokenized):
    N = data.shape[0]  # Get the number of rows
    allWords = []
    if preTokenized != 1 and preTokenized != 0:
        return "Wrong preTokenize input"
    if preTokenized == 1:
        for text in data[column]:
            if isinstance(text, list):  # Ensure text is already tokenized (list of words)
                allWords.extend(text)
            elif isinstance(text, str):  # If still a string, split it as a fallback
                allWords.extend(text.split())
    elif preTokenized == 0:
        for text in data[column]:
            if isinstance(text, str):
                allWords.extend(word_tokenize(text))

    return FreqDist(allWords)

#Alternative? Returns a pandas series, with word and frequency, very fast.
def getFreq(data, column):
    return data[column].str.split().explode().value_counts()

In [21]:
#function for removeing stopwords
def dataStemming(data, column):
    ps = PorterStemmer()
    def dataStemming_help(text):
        if isinstance(text, str):
            return ps.stem(text)
        return text  # Return unchanged if not a string
    data[column] = data[column].parallel_apply(dataStemming_help)  # Apply function
    return data

In [59]:
# One big function to process data:
def processData(data, column):
    def apply_sequential_helper(functions):
        # assume type siganture of functions to be List[f : String -> string ]
        def inner(text):
            for f in functions:
                text = f(text)
            return text
        return inner

    def clean_text_help(text):
        if isinstance(text, str):
            # Remove excess whitespace
            text = re.sub(r"\s+", " ", text).strip()
            #replace dates
            text = re.sub(r"(0[1-9]|[1-2][0-9]|3[0-1])[-/.]?(0[1-9]|1[0-2])[-/.]?([0-9]{2}|[0-9]{4})", "<DATE>", text)  # Replace date type 1
            text = re.sub(r"(0[1-9]|[1-2][0-9]|3[0-1])\s([A-Za-z]{3})\s([0-9]{2}|[0-9]{4})", "<DATE>", text)  # Replace date type 2
            return clean(text, lower=True, no_line_breaks=True, no_punct=True, replace_with_url="<URL>", replace_with_email="<EMAIL>", replace_with_number="<NUM>", replace_with_digit="<NUM>")
        return text  # Return unchanged if not a string

    def tokenize_text_help(text):
        if isinstance(text, str):
            return word_tokenize(text)
        return text  # Return unchanged if not a string

    def remove_stopwords_help(text):
        text = pd.Series(text)
      # text is a list[str]
        stop_words = set(stopwords.words('english'))  # Load stopwords
        #if isinstance(text, str):
        #    return [word for word in text.at[0, 'content'] if not word.lower() in stop_words]
        #return text  # Return unchanged if not a string
        return text[~text.isin(stop_words)]

    ps = PorterStemmer()
    def dataStemming_help(text):
        #if isinstance(text, str):
        #    return ps.stem(text)
        #return text  # Return unchanged if not a string
        return text.apply(ps.stem)

    def type_cleaner(text):
        return text.to_list()

    data[column] = data[column].parallel_apply(apply_sequential_helper(
        [clean_text_help, # str -> str
        tokenize_text_help, # str -> list[str]
        remove_stopwords_help, #series[str] -> series[str]
        dataStemming_help, #series[str] -> series[str]
        type_cleaner # series[str] -> list[str]
    ])).to_list()

    return data

In [60]:
"""
nsdf_cleaned = cleanText(nsdf, 'content')
nsdf_tokenized = tokenizeText(nsdf_cleaned, 'content')                  #tokenizing
nsdf_cleaned_tokenized_nostopwords = remove_stopwords(nsdf_tokenized, 'content')           #removing stopwords
nsdf_preprocessed = dataStemming(nsdf_cleaned_tokenized_nostopwords, 'content')
"""
nsdf_processed = processData(nsdf, 'content')
nsdf_processed.dropna(subset=['content'], inplace=True)  # Drop rows with no content
nsdf_processed.reset_index(drop=True, inplace=True)  # Reset index
print(nsdf_processed.at[0, 'content'])

['sometim', 'power', 'christma', 'make', 'wild', 'wonder', 'thing', 'need', 'believ', 'holi', 'triniti', 'believ', 'posit', 'power', 'good', 'other', 'simpl', 'act', 'give', 'without', 'receiv', 'lost', 'mani', 'us', 'day', 'worri', 'money', 'success', 'hold', 'us', 'back', 'give', 'other', 'need', 'one', 'congreg', 'ohio', 'move', 'action', 'power', 'sermon', 'given', 'church', 'christma', 'eve', 'pastor', 'grand', 'lake', 'unit', 'methodist', 'church', 'celina', 'ohio', 'gave', 'emot', 'sermon', 'import', 'understand', 'messag', 'jesu', 'mani', 'religi', 'peopl', 'messag', 'jesu', 'help', 'other', 'make', 'sure', 'peopl', 'suffer', 'get', 'help', 'need', 'enjoy', 'life', 'littl', 'bit', 'sermon', 'realli', 'generos', 'look', 'like', 'live', 'jesu', 'live', 'long', 'time', 'ago', 'act', 'gener', 'fashion', 'time', 'would', 'gener', 'act', 'look', 'like', 'time', 'focu', 'sermon', 'potenc', 'sermon', 'lost', 'congreg', 'move', 'take', 'action', 'sermon', 'end', 'congreg', 'decid', 'tak

In [61]:
#word frequency pre preprocessing
print("word frequency pre preprocessing")
word_frequency_pre = getFreq(nsdf_raw, 'content').sum()
print(word_frequency_pre)

word frequency pre preprocessing
170462


In [62]:
#word frequency post preprocessing
print("word frequency post preprocessing")
# getfreq assumes different type signature, so we have to do it this way
word_frequency_post = nsdf_processed["content"].explode().value_counts().sum()
print(word_frequency_post)

word frequency post preprocessing
86232


In [63]:
#word frequency post stemming
#print("word frequency post stemming")
#word_frequency_postStem = getFrequency(nsdf_.processed, 'content', 1)
#print(sum(word_frequency_postStem.values()))

In [ ]:
#pre = word_frequency_pre
#post = word_frequency_post
#print("Reduction rate of the vocabulary size after removing stopwords:")
#print(abs(pre -post))
#print("Further  reduction rate of the vocabulary size after stemming")
#postStem = sum(word_frequency_post.values())
#print(abs(post -postStem))

### Task 2

In [ ]:
#load data

fakeNewsCorpus = pd.read_csv(dataPath + "995,000_rows.csv")
#Hva saten er den der unnamed???
print(fakeNewsCorpus.head())
fakeNewsCorpus['content'].duplicated()
news_noDup = fakeNewsCorpus.drop_duplicates(subset=['content'])
#Cleaning
news_processed = processData(news_noDup, 'content')
news_processed.to_csv("../data/news_processed.csv", index=False)

<ipython-input-64-b430c93a156f>:3: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  fakeNewsCorpus = pd.read_csv(dataPath + "995,000_rows.csv")


  Unnamed: 0         id               domain        type  \
0        732  7444726.0   nationalreview.com   political   
1       1348  6213642.0    beforeitsnews.com        fake   
2       7119  3867639.0     dailycurrant.com      satire   
3       1518  9560791.0          nytimes.com    reliable   
4       9345  2059625.0  infiniteunknown.net  conspiracy   

                                                 url  \
0  http://www.nationalreview.com/node/152734/%E2%...   
1  http://beforeitsnews.com/economy/2012/06/the-c...   
2  http://dailycurrant.com/2016/01/18/man-awoken-...   
3  https://query.nytimes.com/gst/fullpage.html?re...   
4  http://www.infiniteunknown.net/2011/09/14/100-...   

                                             content  \
0  Plus one article on Google Plus\n\n(Thanks to ...   
1  The Cost Of The Best Senate Banking Committee ...   
2  Man Awoken From 27-Year Coma Commits Suicide A...   
3  WHEN Julia Geist was asked to draw a picture o...   
4  – 100 Compiled Stud

### Task 3

In [ ]:
print(fakeNewsCorpus.info())   # Check column types and missing values
#fndf = pd.DataFrame(fakeNewsCorpus) # Is this not redundant as pandas automatically reads csv files as dataframes?
fndf = fakeNewsCorpus.reset_index(drop=True)  # Reset index??


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995000 entries, 0 to 994999
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        994999 non-null  object 
 1   id                994993 non-null  object 
 2   domain            994989 non-null  object 
 3   type              947214 non-null  object 
 4   url               994989 non-null  object 
 5   content           994988 non-null  object 
 6   scraped_at        994987 non-null  object 
 7   inserted_at       994987 non-null  object 
 8   updated_at        994987 non-null  object 
 9   title             986394 non-null  object 
 10  authors           552243 non-null  object 
 11  keywords          0 non-null       float64
 12  meta_keywords     956210 non-null  object 
 13  meta_description  469894 non-null  object 
 14  tags              230919 non-null  object 
 15  summary           0 non-null       float64
 16  source            21

In [ ]:
fndf.duplicates()

In [ ]:
print(fndf.loc[fndf['type']==''])

Empty DataFrame
Columns: [Unnamed: 0, id, domain, type, url, content, scraped_at, inserted_at, updated_at, title, authors, keywords, meta_keywords, meta_description, tags, summary, source]
Index: []


### Cleaning and Preprocessing

In [ ]:
print("Pandas DataFrame:")
display(fndf.iloc[0])



Pandas DataFrame:


Unnamed: 0                                                        732
id                                                          7444726.0
domain                                             nationalreview.com
type                                                        political
url                 http://www.nationalreview.com/node/152734/%E2%...
content             Plus one article on Google Plus\n\n(Thanks to ...
scraped_at                                 2017-11-27T01:14:42.983556
inserted_at                                2018-02-08 19:18:34.468038
updated_at                                 2018-02-08 19:18:34.468066
title                                              Iran News Round Up
authors                                                           NaN
keywords                                                          NaN
meta_keywords       ['National Review', 'National Review Online', ...
meta_description                                                  NaN
tags                

In [ ]:
# unique lable values
unique_values = fndf['type'].unique()
print(unique_values)

['political' 'fake' 'satire' 'reliable' 'conspiracy' 'unreliable' 'bias'
 'rumor' 'unknown' nan 'clickbait' 'hate' 'junksci'
 '2018-02-10 13:43:39.521661']


In [ ]:
#hard to know how to classify nan and unknown, so removed for now
fndf = fndf.dropna(subset=['type'])
fndf = fndf.loc[fndf['type']!='unknown']
# Need to reset index

newunique_values = fndf['type'].unique()
print(newunique_values)

['political' 'fake' 'satire' 'reliable' 'conspiracy' 'unreliable' 'bias'
 'rumor' 'clickbait' 'hate' 'junksci' '2018-02-10 13:43:39.521661']


In [ ]:
# groups (reliable) as truenews 1 and (all others) in fakenews 0
    #note this is naive and should be reconsidered later
fndf['type'] = fndf['type'].replace(r'^reliable$', '1', regex=True)  # Only replaces exact 'reliable' with 1
fndf['type'] = fndf['type'].replace(r'^(?!1$).+', '0', regex=True)   # Replace everything except '1' with '0'
#fndf['type'] = fndf['type'].fillna('0')

newunique_values = fndf['type'].unique()
print(newunique_values)
fndf.shape[0]


['0' '1']


903680

In [ ]:
#Cleaning the data set
fndf = fndf['Content'].apply(cleanText).explode()

## Task 4

In [ ]:
# Splitting into test, train and validation
X_train, X_valtest, y_train, y_valtest = train_test_split(fndf['content'], fndf['type'], test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=42)

print("train size:", y_train.shape)
print("val size:", y_val.shape)
print("test size:", y_test.shape)


train size: (722944,)
val size: (90368,)
test size: (90368,)


# Part 2